#Импортирование библиотек

In [1]:
#импортируем необходимые библиотеки

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import precision_score, recall_score

# Загрузка датасетов

In [2]:
#тренировочные датасеты
events_data_train = pd.read_csv('https://stepik.org/media/attachments/course/4852/event_data_train.zip')
submission_data_train = pd.read_csv('https://stepik.org/media/attachments/course/4852/submissions_data_train.zip')

#тестовые датасеты
events_data_test = pd.read_csv('https://stepik.org/media/attachments/course/4852/events_data_test.csv')
submission_data_test = pd.read_csv('https://stepik.org/media/attachments/course/4852/submission_data_test.csv')

#events_data

#step_id - id стэпа
#user_id - анонимизированный id юзера
#timestamp - время наступления события в формате unix date
#action - событие, возможные значения: 
#discovered - пользователь перешел на стэп
#viewed - просмотр шага,
#started_attempt - начало попытки решить шаг, ранее нужно было явно нажать на кнопку - начать решение, перед тем как приступить к решению практического шага
#passed - удачное решение практического шага

#submission_data

#step_id - id стэпа
#timestamp - время отправки решения в формате unix date
#submission_status - статус решения
#user_id - анонимизированный id юзера

#Обработка данных

In [3]:
def to_date_and_day(data: pd.core.frame.DataFrame):
  
  #добавление к датафрэйму столбцы с датой и днём

  data['date'] = pd.to_datetime(data.timestamp, units = 's')
  data['day'] = data['date'].dt.date

  return data

In [4]:
def time_filter(data, count_of_days=2):

  #фильтрация данных с певрого момента, до порогового(count_of_days)

  #определяет ограничение по секудная пребывания
  time_threshold = count_of_days*24*60*60

  #определяет первый момент появления на курсе для каждого юреза
  first_user_time = data.groupby('user_id').agg({'timestamp': 'min'}).reset_index().rename(columns={'timestamp': 'first_user_time'})
  data = data.merge(first_user_time, on='user_id', how = 'outer')

  #перевод id и времени в удобнуб для фильтрации форму
  data['user_time'] = data.user_id.map(str) + '_' + data.timestamp.map(str)
  data['user_time_threshold'] = data.user_id.map(str) + '_' + (data.first_user_time + time_threshold).map(str)

  #фильтрация посредством сравнения строк id и времени каждого действия с пороговым значением для каждого юзера
  data_time_filtered = data.query('user_time < user_time_threshold')

  #удаление промежуточных столбцов сравнения
  data_time_filtered = data_time_filtered.drop(['user_time', 'user_time_threshold', 'first_user_time'], axis=1)

  #возвращает отфильтрованный датафрэйм по пороговому значению дней пребывания на курсе(count_of_days)
  return data_time_filtered

In [5]:
def main_features(events_data: pd.core.frame.DataFrame, submission_data: pd.core.frame.DataFrame):

  #определение основных фичей для дальнейшего анализа
  #действия - из таблицы events_data
  #правильность выполнения задания - из таблицы submission_data

  #определение колличества определенных действий, совершённых каждым юзеров
  users_event_data = pd.pivot_table(data=events_data, 
                                    index='user_id', 
                                    columns='action', 
                                    values='step_id', 
                                    aggfunc='count', 
                                    fill_value=0) \
                                    .reset_index() \
                                    .rename_axis('', axis=1)
  
  #определение колличества правильных и неправильных решения для каждого юзера
  users_submission_data = pd.pivot_table(data=submission_data, 
                                         index='user_id',
                                         columns='submission_status', 
                                         values='step_id', 
                                         aggfunc='count', 
                                         fill_value=0) \
                                         .reset_index() \
                                         .rename_axis('', axis=1)

  #объединение таблиц с полученными фичами
  users_data = pd.merge(users_event_data, users_submission_data, on='user_id', how='outer').fillna(0)

  return users_data

In [6]:
def steps_tried(submissions_data: pd.core.frame.DataFrame):

  #определяет общее количество попоток юзера в выполнении заданий

  steps_tried = submissions_data.groupby('user_id').step_id.nunique() \
                                                          .to_frame() \
                                                          .reset_index() \
                                                          .rename(columns = {'step_id': 'steps_tried'})

  return steps_tried                                                       
  

In [7]:
def correct_ratio(data):
  
  #определяет соотношения правильных решений, ко всем

  data['correct_ratio'] = data['correct']/(data['correct'] + data['wrong'])

  return data


In [8]:
def target(submission_data: pd.core.frame.DataFrame, treshold=40):

  #определение целевой переменной
  #если юзер сделал больше или равное treshold заданий, то считается, что он закончит курс

  #определим колличество верно-решенных задач для каждого юзера
  users_correct = submission_data.query('submission_status == "correct"') \
                                                      .groupby('user_id') \
                                                      .agg({'step_id': 'count'}) \
                                                      .reset_index() \
                                                      .rename({'step_id': 'correct_count'}, axis = 1)

  #сравним с пороговым значением и внесем результат в итоговую таблицу                                                    
  users_correct['passed_course'] = (users_correct.correct_count >= treshold).astype(int)
  users_correct = users_correct.drop('correct_count', axis=1)

  return users_correct

Формирование датафрэймов

In [9]:
def create_train_dataframes(events_data: pd.core.frame.DataFrame, submission_data: pd.core.frame.DataFrame):

  #создает итоговые датафрэймы для построения моделей

  #фильтрует по времени начальные данные
  events_filtered = time_filter(events_data)
  submission_filtered = time_filter(submission_data)

  #создает датафрэйм(из отфильтрованных) с основными фичами
  main_features_data = main_features(events_filtered, submission_filtered)

  #вычисление соотношения правильных ответов ко всем данным
  correct_ratio_data = correct_ratio(main_features_data)

  #вычисление общего числа попыток
  steps_tried_data = steps_tried(submission_filtered)

  #определение целевой переменной
  users_target_feature = target(submission_data)

  #все фичи опрделены, теперь соединяем полученные DataFrames
  result = main_features_data.merge(correct_ratio_data, how='outer')
  result = result.merge(steps_tried_data, how='outer')
  result = result.merge(users_target_feature, how='outer').fillna(0)
  #в переменной result находится итоговый DataFrame со всеми вычесленными фичами и целевой переменной

  #Разделим result на фичи и целевую переменную
  y = result[['passed_course']]
  X = result.drop('passed_course', axis=1)

  return X, y

In [10]:
def create_test_X_dataframe(events_data: pd.core.frame.DataFrame, submission_data: pd.core.frame.DataFrame):

  #создает итоговые датафрэймы для построения моделей

  #фильтрует по времени начальные данные
  events_filtered = time_filter(events_data)
  submission_filtered = time_filter(submission_data)

  #создает датафрэйм(из отфильтрованных) с основными фичами
  main_features_data = main_features(events_filtered, submission_filtered)

  #вычисление соотношения правильных ответов ко всем данным
  correct_ratio_data = correct_ratio(main_features_data)

  #вычисление общего числа попыток
  steps_tried_data = steps_tried(submission_filtered)

  #определение целевой переменной
  users_target_feature = target(submission_data)

  #все фичи опрделены, теперь соединяем полученные DataFrames
  result = main_features_data.merge(correct_ratio_data, how='outer')
  test_data = result.merge(steps_tried_data, how='outer').fillna(0)
  #в переменной result находится итоговый DataFrame со всеми вычесленными фичами


  return test_data
  

# Обучение моделей

In [11]:
def GridRandomForest(X, y, test_data):
  
  #разделение на тренировочную и тестовую выборки
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

  #создание и обучени модели на тренировочных данных
  clf = RandomForestClassifier()
  params = {'max_depth': [6], 'n_estimators': [30]}
  grid_cros_val = GridSearchCV(clf, param_grid=params, cv=6, n_jobs=-1)
  grid_cros_val.fit(X_train, y_train)

  #вывод наилучшей модели
  best_estimator = grid_cros_val.best_estimator_
  print(f'Набор лучших параметров модели:\n {grid_cros_val.best_params_}')

  #предсказание вероятностей отнесения к классам
  y_pred_prob = best_estimator.predict_proba(X_test)

  #непосредственное отнесение к классам в зависимости от порогового значения вероятности
  y_pred = np.where(y_pred_prob[:, 1] >= 0.5, 1, 0)

  #вычисление основных метрик классификации
  accuracy = best_estimator.score(X_test, y_test)
  roc_score = roc_auc_score(y_test, y_pred_prob[:, 1])

  print(f'Accuracy на тестовых данных: {accuracy}',
        f'Roc_score на тестовых данных: {roc_score}', sep='\n')
  #предсказание для итогового теста
  y_pred_prob = best_estimator.predict_proba(test_data)
  result = test_data['user_id'].to_frame()
  result['is_gone'] = y_pred_prob[:, 1]
  result[['user_id', 'is_gone']].to_csv(f'my_predict_{roc_score:.5f}.csv', index=False)
  print(f'Результы записанны в файл my_predict_{roc_score:.5f}.csv')
  


  

#Анализ модели

In [12]:
X, y = create_train_dataframes(events_data_train, submission_data_train)

In [13]:
test_data = create_test_X_dataframe(events_data_test, submission_data_test)

In [14]:
GridRandomForest(X, y, test_data)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Набор лучших параметров модели:
 {'max_depth': 6, 'n_estimators': 30}
Accuracy на тестовых данных: 0.9026165309305146
Roc_score на тестовых данных: 0.8819096903665962
Результы записанны в файл my_predict_0.88191.csv
